# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mata-utu,-13.2816,-176.1745,28.35,76,100,6.64,WF,1702256497
1,1,bodaybo,57.8506,114.1933,-30.12,98,100,0.21,RU,1702256498
2,2,bredasdorp,-34.5322,20.0403,15.14,82,0,1.24,ZA,1702256498
3,3,adamstown,-25.0660,-130.1015,21.29,57,0,3.19,PN,1702256499
4,4,nar'yan-mar,67.6713,53.0870,-22.72,91,100,4.92,RU,1702255914


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df.hvplot.points("Lng", "Lat",
                                      geo = True,
                                      tiles = "OSM",
                                      size = "Humidity",
                                      color="City",
                                      alpha=.5)

# Display the map
humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
#under 32 celsius and 5 m/s wind with some clouds.
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 32) & 
                                   (city_data_df["Wind Speed"] < 5) & 
                                   (city_data_df["Cloudiness"] >10)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,bodaybo,57.8506,114.1933,-30.12,98,100,0.21,RU,1702256498
4,4,nar'yan-mar,67.6713,53.0870,-22.72,91,100,4.92,RU,1702255914
5,5,mhamid,29.8200,-5.7200,15.79,36,11,2.02,MA,1702255919
7,7,keflavik,64.0049,-22.5624,1.24,100,100,4.12,IS,1702256500
10,10,stanley,54.8680,-1.6985,5.71,96,75,3.09,GB,1702256501


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,bodaybo,57.8506,114.1933,-30.12,98,100,0.21,RU,1702256498,
4,4,nar'yan-mar,67.6713,53.0870,-22.72,91,100,4.92,RU,1702255914,
5,5,mhamid,29.8200,-5.7200,15.79,36,11,2.02,MA,1702255919,
7,7,keflavik,64.0049,-22.5624,1.24,100,100,4.12,IS,1702256500,
10,10,stanley,54.8680,-1.6985,5.71,96,75,3.09,GB,1702256501,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
       "categories":"accommodation.hotel",
       "apiKey":geoapify_key,
       "limit":17
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bodaybo - nearest hotel: НОЙ
nar'yan-mar - nearest hotel: Гостиница "Агат"
mhamid - nearest hotel: Hotel Kasbah Azalay
keflavik - nearest hotel: Núpan Deluxe
stanley - nearest hotel: Hotel 52
remire-montjoly - nearest hotel: Complexe Belova
albany - nearest hotel: No hotel found
nao me toque - nearest hotel: ibis
midvagur - nearest hotel: GiljaHome
port douglas - nearest hotel: Mantra Aqueous Port Douglas
pandan - nearest hotel: No hotel found
sinop - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
sucua - nearest hotel: HOTEL BARZA
arica - nearest hotel: Hotel Andalucía 
katsuura - nearest hotel: 潮騒の宿
neiafu - nearest hotel: Port Wine Guest House
cambebba - nearest hotel: Ibis
cajuru - nearest hotel: No hotel found
camana - nearest hotel: Caja Arequipa
georgetown - nearest hotel: Page 63 hostel
berdigestyakh - nearest hotel: No hotel found
inari - nearest hotel: Villa Lanca
tazovsky 

shimoda - nearest hotel: ホテルウラガ
harstad - nearest hotel: Scandic Harstad
menongue - nearest hotel: No hotel found
jayapura - nearest hotel: Hotel Jasmine Jayapura
igrim - nearest hotel: Факел
bubaque - nearest hotel: Chez Julio
prince george - nearest hotel: Travelodge by Wyndham Prince George
malanje - nearest hotel: Hotel PORTVGALIA
mana - nearest hotel: Le Samana
dawson creek - nearest hotel: Comfort Inn
ambovombe - nearest hotel: La source
gueret - nearest hotel: Brit hotel confort
mahina - nearest hotel: Motu Martin
hanzhong - nearest hotel: 汉庭酒店
rakops - nearest hotel: Rakops River Lodge
portland - nearest hotel: the Hoxton
pursat - nearest hotel: No hotel found
heinola - nearest hotel: Tukkijätkä
darlington - nearest hotel: Premier Inn Darlington Town Centre
puerto madero - nearest hotel: No hotel found
shohimardon - nearest hotel: No hotel found
vrangel' - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
platteville - nearest hotel: Holiday Inn Express & Su

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,bodaybo,57.8506,114.1933,-30.12,98,100,0.21,RU,1702256498,НОЙ
4,4,nar'yan-mar,67.6713,53.0870,-22.72,91,100,4.92,RU,1702255914,"Гостиница ""Агат"""
5,5,mhamid,29.8200,-5.7200,15.79,36,11,2.02,MA,1702255919,Hotel Kasbah Azalay
7,7,keflavik,64.0049,-22.5624,1.24,100,100,4.12,IS,1702256500,Núpan Deluxe
10,10,stanley,54.8680,-1.6985,5.71,96,75,3.09,GB,1702256501,Hotel 52
...,...,...,...,...,...,...,...,...,...,...,...
542,542,realeza,-25.7689,-53.5325,18.73,98,13,0.84,BR,1702256657,No hotel found
543,543,newman,37.3138,-121.0208,14.13,43,100,0.89,US,1702256657,No hotel found
547,547,ozernyy,55.5802,32.4127,-4.45,95,100,3.27,RU,1702256659,No hotel found
548,548,arteche,12.2645,125.4048,27.34,87,100,3.99,PH,1702256659,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat",
                                   geo = True,
                                   tiles = "OSM",
                                   size = "Humidity",
                                   color ="City", 
                                   alpha=0.5,
                                   hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)